In [1]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%matplotlib notebook

In [3]:
Game = Regency(True)
clear_output()


In [4]:
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)
Game.take_domain_actions()
Game.take_domain_actions()
Game.take_domain_actions()

Instructions for updating:
Use tf.cast instead.


Time to Finish the Rounditiative -1
0


In [7]:
Game.Troops

,Regent,Provence,Type,Cost,CR,Garrisoned,Home,Injury
0,GG,Rivenrock,Dwarf Guards,2,2.0,0,,0
1,GG,Rivenrock,Dwarf Crossbows,2,2.0,0,,0
2,GG,Rivenrock,Dwarf Guards,2,2.0,0,,0
3,GG,Rivenrock,Dwarf Crossbows,2,2.0,0,,0
4,GG,Rivenrock,Dwarf Guards,2,2.0,0,,0
5,GG,Rivenrock,Dwarf Crossbows,2,2.0,0,,0
6,GG,Rivenrock,Dwarf Guards,2,2.0,0,,0
7,GG,Rivenrock,Dwarf Crossbows,2,2.0,0,,0
8,Sw,Algael,Undead Troops,0,2.0,0,,0
9,Sw,Algael,Undead Troops,0,2.0,0,,0


In [8]:
import pandas as pd
df = pd.concat([Game.Seasons[0]['Actions'][a] for a in Game.Seasons[0]['Actions'].keys()])

print(df[df['Regent']=='GG']['Message'].values[0])

Overthane Grimm Graybeard has ordered the following troops out of garrison:Irregulars from out of "Who cares?"


In [9]:
Game.Projects

,Regent,Project Type,Details,Gold Bars Left
0,JR,Build Ship,"(Keelboat, Gulfport, My Ass)",1
1,Sw,Undead Troops,,2
2,Sw,Undead Troops,,3


In [10]:
Game.Provences[Game.Provences['Provence']=='Gulfport']

,Provence,Domain,Region,Regent,Terrain,Loyalty,Taxation,Population,Magic,Castle,Castle Name,Capital,Position,Contested,Waterway,Brigands
10,Gulfport,Osoerde,The Eastern Marches,JR,Plains,Poor,Severe,3,2,0,,False,"[1515, 2154]",False,True,False
